# Automated ML

Import Dependencies. In the cell below, I import all the dependencies that I will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.dataset import Dataset

from azureml.train.automl import AutoMLConfig

from azureml.widgets import RunDetails

from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig, Model

import json
import requests


## Dataset

### Overview

Employee turnover impacts all organizations. The IBM HR Attrition Case Study aims to identify the factors contributing to employee turnover and forecast those individuals who are more likely to depart from the company.

With a dataset comprising 35 columns, I intend to utilize Microsoft Azure's AutoML feature to develop and evaluate various models for predicting employee turnover. Subsequently, I will deploy the most effective model as a web service for further interaction.

Get data. In the cell below, I write code to access the data I will be using in this project.

In [2]:
ws = Workspace.from_config(path="/")

experiment_name = 'AutoML'

experiment=Experiment(ws, experiment_name)

In [3]:
found = False
key = "Employee Attrition"
description_text = "IBM HR Analytics Employee Attrition & Performance"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        url = "https://raw.githubusercontent.com/eljandoubi/Azure-Machine-Learning-Engineer/main/attrition-dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(url)        
        dataset = dataset.register(workspace=ws,
                                   name=key,description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [5]:
cluster_name = "automl-vs-hpyer"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target, using it!')
    
except ComputeTargetException:
    print('Creating a new compute target!')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',min_nodes=1, max_nodes=8)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)
 
cpu_cluster.get_status().serialize()

Found existing compute target, using it!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


{'currentNodeCount': 1,
 'targetNodeCount': 1,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 0,
  'idleNodeCount': 1,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2024-02-17T11:46:54.437000+00:00',
 'errors': None,
 'creationTime': '2024-02-17T08:24:17.056201+00:00',
 'modifiedTime': '2024-02-17T08:24:26.830859+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 1,
  'maxNodeCount': 8,
  'nodeIdleTimeBeforeScaleDown': 'PT1800S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_D2_V2'}

## AutoML Configuration

This task involves binary classification, where the target variable is `Attrition`, with possible outcomes being `True` or `False`. The experiment's timeout duration is set to 30 minutes, allowing for a maximum of 8 concurrent iterations. The primary evaluation metric used is `AUC_weighted`.

In [6]:
automl_settings = automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 8,
    "primary_metric" : 'AUC_weighted'
}

# Automl config
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Attrition",   
                             path = './automl',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                             )

In [7]:
autoML_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML,AutoML_ec394bf3-d2da-4664-bd5c-291e6a621909,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In the cell below, i use the `RunDetails` widget to show the different experiments.

In [8]:
autoML_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
AutoML,AutoML_ec394bf3-d2da-4664-bd5c-291e6a621909,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number o

{'runId': 'AutoML_ec394bf3-d2da-4664-bd5c-291e6a621909',
 'target': 'automl-vs-hpyer',
 'status': 'Completed',
 'startTimeUtc': '2024-02-17T12:05:28.262003Z',
 'endTimeUtc': '2024-02-17T12:26:15.75824Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'automl-vs-hpyer',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML","subscription_id":"9e65f93e-bdd8-437b-b1e8-0647cd6098f7","resource_group":"aml-quickstarts-253507","workspace_name":"quick-starts-ws-253507","region":"southcentralus","compute_target":"automl-vs-h

In [9]:
RunDetails(autoML_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run, fitted_model = autoML_run.get_output()
best_run_metrics = best_run.get_metrics()

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

In [11]:
fitted_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('40', Pipeline(memory=None, steps=[('truncatedsvdwrapper', TruncatedSVDWrapper(n_components=0.9005263157894737, random_state=None)), ('logisticregression', LogisticRegression(C=0.08685113737513521, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, solver='newton-cg', tol=0.0001, verbose=0, war

In [12]:
best_run_metrics

{'balanced_accuracy': 0.6824076854321671,
 'f1_score_micro': 0.8727891156462585,
 'f1_score_weighted': 0.8576762619515105,
 'accuracy': 0.8727891156462585,
 'weighted_accuracy': 0.9433883210429864,
 'AUC_macro': 0.8405155094445275,
 'norm_macro_recall': 0.36481537086433424,
 'recall_score_micro': 0.8727891156462585,
 'average_precision_score_macro': 0.7815160961279656,
 'log_loss': 0.34647340142342453,
 'average_precision_score_weighted': 0.8953248972888993,
 'precision_score_micro': 0.8727891156462585,
 'precision_score_macro': 0.795254214191962,
 'matthews_correlation': 0.46216793066410905,
 'average_precision_score_micro': 0.9267864187864281,
 'AUC_micro': 0.9351728446480633,
 'recall_score_weighted': 0.8727891156462585,
 'AUC_weighted': 0.8405155094445275,
 'f1_score_macro': 0.7149398593537789,
 'precision_score_weighted': 0.860754509263546,
 'recall_score_macro': 0.6824076854321671,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_ec394bf3-d2da-4664-bd5c-291e6a621909

In [13]:
print('Best Run Id: ', best_run.id)
print('AUC_weighted of Best Run is:', best_run_metrics['AUC_weighted'])

Best Run Id:  AutoML_ec394bf3-d2da-4664-bd5c-291e6a621909_63
AUC_weighted of Best Run is: 0.8405155094445275


In [14]:
automodel = best_run.register_model(model_name='automl_model', 
                                    model_path='outputs/model.pkl',
                                    tags={'Method':'AutoML'},
                                    properties={'AUC_weighted': best_run_metrics['AUC_weighted']})

automodel

Model(workspace=Workspace.create(name='quick-starts-ws-253507', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-253507'), name=automl_model, id=automl_model:1, version=1, tags={'Method': 'AutoML'}, properties={'AUC_weighted': '0.8405155094445275'})

## Model Deployment

In the cell below, I register the model, create an inference config and deploy the model as a web service.

In [15]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [16]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               description='Predict employee attrition')

In [17]:
inference_config = InferenceConfig(entry_script="score.py", environment=best_run.get_environment())

service = Model.deploy(workspace=ws, 
                       name='automl-webservice', 
                       models=[automodel], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

In [18]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-17 12:27:55+00:00 Creating Container Registry if not exists.
2024-02-17 12:27:55+00:00 Registering the environment.
2024-02-17 12:27:56+00:00 Use the existing image.
2024-02-17 12:27:56+00:00 Submitting deployment to compute.
2024-02-17 12:28:02+00:00 Checking the status of deployment automl-webservice..
2024-02-17 12:30:16+00:00 Checking the status of inference endpoint automl-webservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [19]:
service.update(enable_app_insights=True)

In [20]:
print("Service State: ",service.state)
scoring_uri = service.scoring_uri
scoring_uri

Service State:  Healthy


'http://881e09a8-e231-4a7f-91b4-8505c7be786a.southcentralus.azurecontainer.io/score'

In the cell below, I send a request to the web service I deployed to test it.

In [21]:
test_df = df.sample(4)
test_df.pop('Attrition')

input_data = json.dumps({'data': test_df.to_dict(orient='records')})
input_data

'{"data": [{"Age": 24, "BusinessTravel": "Travel_Rarely", "DailyRate": 1127, "Department": "Research & Development", "DistanceFromHome": 18, "Education": 1, "EducationField": "Life Sciences", "EmployeeCount": 1, "EmployeeNumber": 150, "EnvironmentSatisfaction": 2, "Gender": "Male", "HourlyRate": 52, "JobInvolvement": 3, "JobLevel": 1, "JobRole": "Laboratory Technician", "JobSatisfaction": 3, "MaritalStatus": "Married", "MonthlyIncome": 2774, "MonthlyRate": 13257, "NumCompaniesWorked": 0, "Over18": true, "OverTime": false, "PercentSalaryHike": 12, "PerformanceRating": 3, "RelationshipSatisfaction": 3, "StandardHours": 80, "StockOptionLevel": 1, "TotalWorkingYears": 6, "TrainingTimesLastYear": 2, "WorkLifeBalance": 3, "YearsAtCompany": 5, "YearsInCurrentRole": 3, "YearsSinceLastPromotion": 1, "YearsWithCurrManager": 2}, {"Age": 40, "BusinessTravel": "Travel_Rarely", "DailyRate": 444, "Department": "Sales", "DistanceFromHome": 2, "Education": 2, "EducationField": "Marketing", "EmployeeCou

In [22]:
headers = {'Content-Type': 'application/json'}

resp = requests.post(scoring_uri, input_data, headers=headers)

resp.text

'"{\\"result\\": [false, false, false, true]}"'

In the cell below, I print the logs of the web service and delete the service

In [23]:
service.get_logs()

'2024-02-17T12:30:05,384467900+00:00 - rsyslog/run \n2024-02-17T12:30:05,413223200+00:00 - gunicorn/run \n2024-02-17T12:30:05,425447300+00:00 | gunicorn/run | \n2024-02-17T12:30:05,438350900+00:00 | gunicorn/run | ###############################################\n2024-02-17T12:30:05,447018100+00:00 | gunicorn/run | AzureML Container Runtime Information\n2024-02-17T12:30:05,452018500+00:00 - nginx/run \n2024-02-17T12:30:05,459927600+00:00 | gunicorn/run | ###############################################\n2024-02-17T12:30:05,477602400+00:00 | gunicorn/run | \n2024-02-17T12:30:05,499712500+00:00 | gunicorn/run | \n2024-02-17T12:30:05,520752700+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2\n2024-02-17T12:30:05,534907800+00:00 | gunicorn/run | \n2024-02-17T12:30:05,550774500+00:00 | gunicorn/run | \n2024-02-17T12:30:05,560592800+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

In [24]:
service.delete()